##### Copyright 2019 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# tf.data を使って CSV をロードする

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/csv"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ja/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

このチュートリアルでは、CSV データを `tf.data.Dataset` にロードする手法の例を示します。

このチュートリアルで使われているデータはタイタニック号の乗客リストから取られたものです。乗客が生き残る可能性を、年齢、性別、チケットの等級、そして乗客が一人で旅行しているか否かといった特性から予測することを試みます。

## 設定

In [0]:
try:
  !pip install tf-nightly-2.0-preview
except Exception:
  pass


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [0]:
# numpy の値を読みやすくする
np.set_printoptions(precision=3, suppress=True)

## データのロード

それではいつものように、扱っている CSV ファイルの先頭を見てみましょう。

In [0]:
!head {train_file_path}

ご覧のように、CSV の列にはラベルが付いています。後ほど必要になるので、ファイルから読み出しておきましょう。

In [0]:
# 入力ファイル中の CSV 列
with open(train_file_path, 'r') as f:
    names_row = f.readline()


CSV_COLUMNS = names_row.rstrip('\n').split(',')
print(CSV_COLUMNS)

 データセットコンストラクタはこれらのラベルを自動的にピックアップします。

使用するファイルの1行目に列名がない場合、`make_csv_dataset` 関数の `column_names` 引数に文字列のリストとして渡します。

```python

CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

dataset = tf.data.experimental.make_csv_dataset(
     ...,
     column_names=CSV_COLUMNS,
     ...)
  
```


この例では使用可能な列をすべて使うことになります。データセットから列を除く必要がある場合には、使用したい列だけを含むリストを作り、コンストラクタの（オプションである）`select_columns` 引数として渡します。


```python

drop_columns = ['fare', 'embark_town']
columns_to_use = [col for col in CSV_COLUMNS if col not in drop_columns]

dataset = tf.data.experimental.make_csv_dataset(
  ...,
  select_columns = columns_to_use, 
  ...)

```

各サンプルのラベルとなる列を特定し、それが何であるかを示す必要があります。

In [0]:
LABELS = [0, 1]
LABEL_COLUMN = 'survived'

FEATURE_COLUMNS = [column for column in CSV_COLUMNS if column != LABEL_COLUMN]

コンストラクタの引数の値が揃ったので、ファイルから CSV データを読み込みデータセットを作ることにしましょう。

（完全なドキュメントは、`tf.data.experimental.make_csv_dataset` を参照してください）

In [0]:
def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # 見やすく表示するために意図して小さく設定しています
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

データセットを構成する要素は、(複数のサンプル, 複数のラベル)の形のタプルとして表されるバッチです。サンプル中のデータは（行ベースのテンソルではなく）列ベースのテンソルとして構成され、それぞれはバッチサイズ（このケースでは12個）の要素が含まれます。

実際に見てみましょう。

In [0]:
examples, labels = next(iter(raw_train_data)) # 最初のバッチのみ
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

## データの前処理

### カテゴリデータ

この CSV データ中のいくつかの列はカテゴリ列です。つまり、その中身は、限られた選択肢の中のひとつである必要があります。

この CSV では、これらの選択肢はテキストとして表現されています。このテキストは、モデルの訓練を行えるように、数字に変換する必要があります。これをやりやすくするため、カテゴリ列のリストとその選択肢のリストを作成する必要があります。

In [0]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


カテゴリ値のテンソルを受け取り、それを値の名前のリストとマッチングして、さらにワンホット・エンコーディングを行う関数を書きます。

In [0]:
def process_categorical_data(data, categories):
  """カテゴリ値を表すワンホット・エンコーディングされたテンソルを返す"""
  
  # 最初の ' ' を取り除く
  data = tf.strings.regex_replace(data, '^ ', '')
  # 最後の '.' を取り除く
  data = tf.strings.regex_replace(data, r'\.$', '')
  
  # ワンホット・エンコーディング
  # data を1次元（リスト）から2次元（要素が1個のリストのリスト）にリシェープ
  data = tf.reshape(data, [-1, 1])
  # それぞれの要素について、カテゴリ数の長さの真偽値のリストで、
  # 要素とカテゴリのラベルが一致したところが True となるものを作成
  data = categories == data
  # 真偽値を浮動小数点数にキャスト
  data = tf.cast(data, tf.float32)
  
  # エンコーディング全体を次の1行に収めることもできる：
  # data = tf.cast(categories == tf.reshape(data, [-1, 1]), tf.float32)
  return data

この処理を可視化するため、最初のバッチからカテゴリ列のテンソル1つを取り出し、処理を行い、前後の状態を示します。

In [0]:
class_tensor = examples['class']
class_tensor

In [0]:
class_categories = CATEGORIES['class']
class_categories

In [0]:
processed_class = process_categorical_data(class_tensor, class_categories)
processed_class

2つの入力の長さと、出力の形状の関係に注目してください。

In [0]:
print("Size of batch: ", len(class_tensor.numpy()))
print("Number of category labels: ", len(class_categories))
print("Shape of one-hot encoded tensor: ", processed_class.shape)

### 連続データ

連続データは値が0と1の間にになるように標準化する必要があります。これを行うために、それぞれの値を、1を列値の平均の2倍で割ったものを掛ける関数を書きます。

この関数は、データの2次元のテンソルへのリシェープも行います。

In [0]:
def process_continuous_data(data, mean):
  # data の標準化
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])

この計算を行うためには、列値の平均が必要です。現実には、この値を計算する必要があるのは明らかですが、この例のために値を示します。

In [0]:
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

前と同様に、この関数が実際に何をしているかを見るため、連続値のテンソルを1つ取り、処理前と処理後を見てみます。

In [0]:
age_tensor = examples['age']
age_tensor

In [0]:
process_continuous_data(age_tensor, MEANS['age'])

### データの前処理

これらの前処理のタスクを1つの関数にまとめ、データセット内のバッチにマッピングできるようにします。

In [0]:
def preprocess(features, labels):
  
  # カテゴリ特徴量の処理
  for feature in CATEGORIES.keys():
    features[feature] = process_categorical_data(features[feature],
                                                 CATEGORIES[feature])

  # 連続特徴量の処理
  for feature in MEANS.keys():
    features[feature] = process_continuous_data(features[feature],
                                                MEANS[feature])
  
  # 特徴量を1つのテンソルに組み立てる
  features = tf.concat([features[column] for column in FEATURE_COLUMNS], 1)
  
  return features, labels



次に、 `tf.Dataset.map` 関数を使って適用し、過学習を防ぐためにデータセットをシャッフルします。

In [0]:
train_data = raw_train_data.map(preprocess).shuffle(500)
test_data = raw_test_data.map(preprocess)

サンプル1個がどうなっているか見てみましょう。

In [0]:
examples, labels = next(iter(train_data))

examples, labels

このサンプルは、（バッチサイズである）12個のアイテムをもつ2次元の配列からできています。アイテムそれぞれは、元の CSV ファイルの1行を表しています。ラベルは12個の値をもつ1次元のテンソルです。

## モデルの構築

この例では、[Keras Functional API](https://www.tensorflow.org/guide/keras/functional) を使用し、単純なモデルを構築するために `get_model` コンストラクタでラッピングしています。

In [0]:
def get_model(input_dim, hidden_units=[100]):
  """複数の層を持つ Keras モデルを作成

  引数:
    input_dim: (int) バッチ中のアイテムの形状
    labels_dim: (int) ラベルの形状
    hidden_units: [int] DNN の層のサイズ（入力層が先）
    learning_rate: (float) オプティマイザの学習率
    
  戻り値:
    Keras モデル
  """

  inputs = tf.keras.Input(shape=(input_dim,))
  x = inputs

  for units in hidden_units:
    x = tf.keras.layers.Dense(units, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(inputs, outputs)
 
  return model

`get_model` コンストラクタは入力データの形状（バッチサイズを除く）を知っている必要があります。

In [0]:
input_shape, output_shape = train_data.output_shapes

input_dimension = input_shape.dims[1] # [0] はバッチサイズ

## 訓練、評価、そして予測

これでモデルをインスタンス化し、訓練することができます。

In [0]:
model = get_model(input_dimension)
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.fit(train_data, epochs=20)

モデルの訓練が終わったら、`test_data` データセットでの正解率をチェックできます。

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

単一のバッチ、または、バッチからなるデータセットのラベルを推論する場合には、`tf.keras.Model.predict` を使います。

In [0]:
predictions = model.predict(test_data)

# 結果のいくつかを表示
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))
